In [2]:
import os
#import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from download_imdb import maybe_download_imdb
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
# EXAMPLE
vec1 = CountVectorizer(ngram_range=(1, 1), 
                      strip_accents='unicode')
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'Here is the first or the third?'
]
new_corp = vec1.fit_transform(corpus)
print(type(new_corp))
new_corp.toarray()
print(vec1.get_feature_names())

<class 'scipy.sparse.csr.csr_matrix'>
['and', 'document', 'first', 'here', 'is', 'one', 'or', 'second', 'the', 'third', 'this']


In [4]:
new_corp.toarray()

array([[0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0],
       [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1],
       [0, 0, 1, 1, 1, 0, 1, 0, 2, 1, 0]], dtype=int64)

In [53]:
stop_string = """
's
a
able
about
above
abst
accordance
according
accordingly
across
act
actually
added
adj
affected
affecting
affects
after
afterwards
again
against
ah
all
almost
alone
along
already
also
although
always
am
among
amongst
an
and
announce
another
any
anybody
anyhow
anymore
anyone
anything
anyway
anyways
anywhere
apparently
approximately
are
aren
arent
arise
around
as
aside
ask
asking
at
auth
available
away
awfully
back
be
became
because
become
becomes
becoming
been
before
beforehand
begin
beginning
beginnings
begins
behind
being
believe
below
beside
besides
between
beyond
biol
both
brief
briefly
but
by
came
can
cannot
can't
cause
causes
certain
certainly
co
com
come
comes
contain
containing
contains
could
couldnt
date
did
didn't
different
do
does
doesn't
doing
done
don't
down
downwards
due
during
each
ed
edu
effect
eg
eight
eighty
either
else
elsewhere
end
ending
enough
especially
et
et-al
etc
even
ever
every
everybody
everyone
everything
everywhere
ex
except
far
few
fifth
first
five
fix
followed
following
follows
for
former
formerly
forth
found
four
from
further
furthermore
gave
get
gets
getting
give
given
gives
giving
go
goes
gone
got
gotten
had
happens
hardly
has
hasn't
have
haven't
having
he
hed
hence
her
here
hereafter
hereby
herein
heres
hereupon
hers
herself
hes
hi
hid
him
himself
his
hither
home
how
howbeit
however
hundred
i
id
ie
if
i'll
im
immediate
immediately
importance
important
in
inc
indeed
index
information
instead
into
invention
inward
is
isn't
it
itd
it'll
its
itself
i've
just
keep
keeps
kept
kg
km
know
known
knows
largely
last
lately
later
latter
latterly
least
less
lest
let
lets
like
liked
likely
line
little
'll
look
looking
looks
ltd
made
mainly
make
makes
many
may
maybe
me
mean
means
meantime
meanwhile
merely
mg
might
million
miss
ml
more
moreover
most
mostly
mr
mrs
much
mug
must
my
myself
name
namely
nay
near
nearly
necessarily
necessary
need
needs
neither
never
nevertheless
new
next
nine
ninety
no
nobody
non
none
nonetheless
noone
nor
normally
nos
not
n't
noted
nothing
now
nowhere
obtain
obtained
obviously
of
off
often
oh
ok
okay
old
omitted
on
once
one
ones
only
onto
or
ord
other
others
otherwise
ought
our
ours
ourselves
out
outside
over
overall
owing
own
p
page
pages
part
particular
particularly
past
per
perhaps
placed
please
plus
poorly
possible
possibly
potentially
predominantly
present
previously
primarily
probably
promptly
proud
provides
put
que
quickly
quite
ran
rather
readily
really
recent
recently
ref
refs
regarding
regardless
regards
related
relatively
research
respectively
resulted
resulting
results
right
run
said
same
saw
say
saying
says
sec
section
see
seeing
seem
seemed
seeming
seems
seen
self
selves
sent
seven
several
shall
she
shed
she'll
shes
should
shouldn't
show
showed
shown
showns
shows
significant
significantly
similar
similarly
since
six
slightly
so
some
somebody
somehow
someone
somethan
something
sometime
sometimes
somewhat
somewhere
soon
sorry
specifically
specified
specify
specifying
still
stop
strongly
sub
substantially
successfully
such
sufficiently
suggest
sup
sure
"""

In [54]:
stops = set(stop_string.strip().splitlines())

In [ ]:
{'POSITIVE':True} {'POSITIVE':False}

In [23]:
def clean_y(y_raw):
    """
    Strips out True and False values from the dicts in y_raw. Makes and returns an np array of dtype=int from the values
    """
    y_clean =np.array([item["POSITIVE"] for item in y_raw], dtype=int)
    return y_clean


filename = 'imdb_thinc_data.pickle'
filepath = os.path.join('..','data','thinc',filename)
_, _, X_test_raw, y_test_raw =maybe_download_imdb(filepath)


In [58]:
X_array_raw= [X_test_raw[5],X_test_raw[6],X_test_raw[7]]
X_array_raw

["No Fireworks Despite Violent Action.\n\n\n\nScience fiction films that reflect quality are scarce indeed, largely because transposal of imaginative themes from the genre to the screen too often falls short of effective execution as a result of insufficient funding or inadequate invention, and unfortunately for its producers, this work is lacking on both counts, woefully so in the case of the latter. With essentially no budget with which to operate, it is a grave mistake to attempt the depiction of such a gamut of events as those within this scenario and, in particular, special effects of space opera warfare which appear only clownish, while seeds from the scriptors' imagination lie fallow due to some of the most fatuous misunderstanding of basic scientific principles to be found. Among these are frequent firing of weapons within a sealed environment, and a wayward law of gravity which enables freedom of movement of cast members while inanimate objects float weightlessly, but it is ea

In [62]:
#use this as supplied tokenizer
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
    
vec = CountVectorizer(ngram_range=(1, 1), 
                      stop_words=stops, 
                      strip_accents='unicode',
                     token_pattern=r'\b[a-zA-Z]{3,}\b',
                     tokenizer=LemmaTokenizer())

counts = vec.fit_transform(X_array_raw)

In [63]:
len(vec.get_feature_names())

201

In [66]:
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(counts)